In [3]:
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
from fastprogress.fastprogress import progress_bar
import pandas as pd

cwd = os.getcwd()
parentDir = os.path.dirname( cwd )
sys.path.append(parentDir)

In [19]:
# Load cylinder data
data_root = os.path.join(parentDir, 'data', 'raw', 'QSM', 'detailed')
noise_data_root = os.path.join(parentDir, 'data', 'noised', 'cloud')
QSMs = [os.path.join(data_root, path) for path in os.listdir(data_root) if path.endswith('.csv')]

for qsm in QSMs:
    # Extract filename from QSM path
    qsm_filename = os.path.basename(qsm)  # Example: "33_22_000000.csv"
    base_name = "_".join(qsm_filename.split("_")[:2])  # Extracts "33_22"
    # Define save paths
    npy_path = os.path.join(noise_data_root, f"{base_name}.npy")
    txt_path = os.path.join(noise_data_root, f"{base_name}.txt")

    cylinders = pd.read_csv(qsm)
    cylinders.columns = cylinders.columns.str.strip()

    # Extract cylinder parameters
    start = cylinders[['startX', 'startY', 'startZ']].values  # (N, 3)
    end = cylinders[['endX', 'endY', 'endZ']].values  # (N, 3)
    radius = cylinders['radius'].values  # (N,)
    axis = end - start
    axis_length = np.linalg.norm(axis, axis=1)  # (N,)
    axis_unit = axis / axis_length[:, None]  # Normalize per row

    # Compute number of points per cylinder
    density = 40  # Points per m²
    angles_per_cm = (2 * np.pi * radius * density).astype(int)
    heights_per_cm = (axis_length * density).astype(int)
    num_points = (angles_per_cm * heights_per_cm)  # (N,)

    # Create index array to map points to cylinders
    cylinder_ids = np.repeat(np.arange(len(cylinders)), num_points)

    # Generate random theta and height in a fully vectorized way
    theta = np.random.uniform(0, 2 * np.pi, size=cylinder_ids.shape)
    z = np.random.uniform(0, axis_length[cylinder_ids], size=cylinder_ids.shape)

    # Add random noise to the radius
    noise = np.random.lognormal(mean=-3, sigma=0.85, size=cylinder_ids.shape)  # Adjust parameters
    r_noisy = radius[cylinder_ids] + noise

    # Convert to local Cartesian coordinates
    x_local = r_noisy * np.cos(theta)
    y_local = r_noisy * np.sin(theta)
    z_local = z
    points_local = np.stack([x_local, y_local, z_local], axis=1)  # (Total_points, 3)

    # Compute rotation matrices for all cylinders
    z_axis = np.array([0, 0, 1])  # Local cylinder z-axis
    v = np.cross(z_axis, axis_unit)  # (N, 3)
    s = np.linalg.norm(v, axis=1)
    c = np.dot(z_axis, axis_unit.T)  # (N,)

    # Handle edge cases where v is zero (axis already aligned)
    v[s.flatten() == 0] = np.array([1, 0, 0])  # Set to arbitrary perpendicular vector

    # Compute skew-symmetric cross-product matrices Vx
    Vx = np.zeros((len(axis_unit), 3, 3))
    Vx[:, 0, 1] = -v[:, 2]
    Vx[:, 0, 2] = v[:, 1]
    Vx[:, 1, 0] = v[:, 2]
    Vx[:, 1, 2] = -v[:, 0]
    Vx[:, 2, 0] = -v[:, 1]
    Vx[:, 2, 1] = v[:, 0]

    # Compute rotation matrices using Rodrigues' formula
    I = np.eye(3)[None, :, :]  # Shape (1, 3, 3) to broadcast with Vx
    R = I + Vx + np.einsum('nij,njk->nik', Vx, Vx) * ((1 - c) / (s ** 2 + 1e-8))[:, None, None]

    # Rotate points
    points_rotated = np.einsum('nij,nj->ni', R[cylinder_ids], points_local)

    # Translate to world coordinates
    points_world = points_rotated + start[cylinder_ids]

    # Save as .npy (binary format for fast loading)
    np.save(npy_path, points_world)

    # # Save as .txt (for readability)
    # np.savetxt(txt_path, points_world, fmt="%.6f", delimiter=" ")

211384
219856
36262
187879
151863
60328
124658
482599
141084
227931
222099
182083
66309
110983
347655
21772
134258
420382
364419
169854
726127
130456
136522
6260
18718
216952
185613
13177
165453
129837
38052
93333
148957
506960
220852
54645
58999
184455
93471
204926
83297
154408
125395
172883
67668
83368
146936
152094
137259
169473
112971
208009
72596
126545
165629
108812
432368
90422
6463
41478
37294
129594
74323
177856
247604
79552
77465
606669
159002
114468
164429
114882
61218
26490
71751
100731
61098
147682
147677
15718
37906
152005
53271
97860
174491
89641
43944
41601
157929
37721
316846
55037
21090
21313
4314
78735
8996
86028
102516
19827
12593
112151
17365
34061
29262
8711
74668
66493
29354
28152
93725
25873
51886
19435
75900
16245
168707
39328
153658
41902
22880
118282
75919
126288
17807
22920
91661
41433
151593
117433
53252
59687
83645
480911
108407
18118
304708
157861
53208
97910
81869
26577
70764
191415
38066
173492
165223
143392
8213
116264
26096
59992
113969
101347
149785


In [10]:
print( points_world[:60] )

[[-49.3813918    8.69853237   7.55292783]
 [-49.76524389   8.3645982    7.66724336]
 [-49.40443328   8.77742025   7.33987948]
 [-49.54432865   9.01309039   7.50043506]
 [-49.71022992   8.48031459   7.39716825]
 [-49.20343864   8.6361621    7.31371414]
 [-49.70528053   8.47572242   7.55473411]
 [-50.0720478    8.7206661    7.52622043]
 [-49.74846854   8.34625552   7.38206475]
 [-49.27004376   8.02152777   7.40794045]
 [-49.45052644   8.8135274    7.57458054]
 [-49.50375432   8.13399552   7.4210084 ]
 [-50.05903914   8.84389283   7.62094435]
 [-49.36608865   7.67926799   7.43654598]
 [-50.00954123   8.83561014   7.5083793 ]
 [-49.83088538   8.52873481   7.5757751 ]
 [-49.84498375   8.8237914    7.74271778]
 [-49.82444809   8.36260587   7.47385224]
 [-50.22465071   8.56037792   7.66528739]
 [-49.96054838   8.87730687   7.42037649]
 [-50.52033527   8.86796872   7.6531975 ]
 [-49.37749928   9.00999512   7.64070969]
 [-49.32957442   8.78792814   7.60119143]
 [-50.08352666   8.51400255   7.59